In [8]:
# Visualising the data
# • Plot a probability density histogram for each type of stimulus. This height of each bar
# should be the proportion of responses divided by the width of the column.

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob

# Load the data
data_path= "Data/Part_2/"
dfs = []
for file in glob.glob('Data/Part_2/*.txt'):
  dfs.append(pd.DataFrame(np.loadtxt(file)))
df = pd.concat(dfs)